In [1]:
import pandas as pd
from tqdm import tqdm

from generators.naive_random_walk_generator import NaiveRandomWalkGenerator
from generators.naive_reverse_walk_generator import NaiveReverseWalkGenerator
from generators.heuristic_reverse_walk_generator import HeuristicReverseWalkGenerator
from generators.es_generator import EsGenerator

from solvers.astar_solver import AStarSolver

TEST_SIZE = 1000

df = pd.DataFrame(columns=['algorithm', 'solution length', 'nodes expanded', 'shannon entropy', 'fragmentation'])
solver = AStarSolver()

In [2]:
from game.logic import Logic, Flask
import math
from collections import Counter
from typing import List


def shannon_entropy(logic: Logic) -> float:
    result = 0
    for flask in logic.board:
        balls = flask.balls
        if not balls:
            continue

        counts = Counter(balls)
        total = len(balls)
        entropy = 0.0

        for count in counts.values():
            p = count / total
            entropy -= p * math.log2(p)

        result += entropy

    return result / len(logic.board)

def fragmentation_score(logic: Logic) -> float:
    result = 0
    for flask in logic.board:
        balls = flask.balls

        if len(balls) < 2:
            continue

        changes = 0
        current_color = balls[0]

        for color in balls[1:]:
            if color != current_color:
                changes += 1
                current_color = color

        result += changes

    return result / len(logic.board)

In [3]:
generator = NaiveRandomWalkGenerator(solver, num_flasks=22, flask_size=4, num_colors=20)

avg_sol_len = 0
avg_nodes_expanded = 0
avg_entropy = 0
avg_frag = 0

for _ in tqdm(range(TEST_SIZE)):
    logic = generator.generate()
    _, nodes_explored, path = solver.solve(logic)
    avg_sol_len += len(path)
    avg_nodes_expanded += nodes_explored
    avg_entropy += shannon_entropy(logic)
    avg_frag += fragmentation_score(logic)

avg_sol_len /= TEST_SIZE
avg_nodes_expanded /= TEST_SIZE
avg_entropy /= TEST_SIZE
avg_frag /= TEST_SIZE

print(avg_entropy, avg_frag, avg_nodes_expanded, avg_sol_len)
df.loc[len(df)] = {
    'algorithm': "NaiveRandomWalk", 
    'solution length': avg_sol_len,
    'nodes expanded': avg_nodes_expanded,
    'shannon entropy': avg_entropy,
    'fragmentation': avg_frag
}

100%|██████████| 1000/1000 [00:14<00:00, 67.18it/s]

0.0 0.0 3.239 2.228


In [4]:
generator = NaiveReverseWalkGenerator(solver, num_flasks=22, flask_size=4, num_colors=20)

avg_sol_len = 0
avg_nodes_expanded = 0
avg_entropy = 0
avg_frag = 0

for _ in tqdm(range(TEST_SIZE)):
    logic = generator.generate()
    _, nodes_explored, path = solver.solve(logic)
    avg_sol_len += len(path)
    avg_nodes_expanded += nodes_explored
    avg_entropy += shannon_entropy(logic)
    avg_frag += fragmentation_score(logic)

avg_sol_len /= TEST_SIZE
avg_nodes_expanded /= TEST_SIZE
avg_entropy /= TEST_SIZE
avg_frag /= TEST_SIZE

print(avg_entropy, avg_frag, avg_nodes_expanded, avg_sol_len)
df.loc[len(df)] = {
    'algorithm': "NaiveReverseWalk", 
    'solution length': avg_sol_len,
    'nodes expanded': avg_nodes_expanded,
    'shannon entropy': avg_entropy,
    'fragmentation': avg_frag
}

100%|██████████| 1000/1000 [03:34<00:00,  4.67it/s]

1.3128097267276493 1.8124545454545462 1696.912 60.573


In [5]:
generator = HeuristicReverseWalkGenerator(solver, num_flasks=22, flask_size=4, num_colors=20)

avg_sol_len = 0
avg_nodes_expanded = 0
avg_entropy = 0
avg_frag = 0

for _ in tqdm(range(TEST_SIZE)):
    logic = generator.generate()
    _, nodes_explored, path = solver.solve(logic)
    avg_sol_len += len(path)
    avg_nodes_expanded += nodes_explored
    avg_entropy += shannon_entropy(logic)
    avg_frag += fragmentation_score(logic)

avg_sol_len /= TEST_SIZE
avg_nodes_expanded /= TEST_SIZE
avg_entropy /= TEST_SIZE
avg_frag /= TEST_SIZE

print(avg_entropy, avg_frag, avg_nodes_expanded, avg_sol_len)
df.loc[len(df)] = {
    'algorithm': "HeuristicReverseWalk", 
    'solution length': avg_sol_len,
    'nodes expanded': avg_nodes_expanded,
    'shannon entropy': avg_entropy,
    'fragmentation': avg_frag
}

100%|██████████| 1000/1000 [03:14<00:00,  5.15it/s]

1.3203154343997923 1.8221363636363637 1401.923 60.857


In [7]:
generator = EsGenerator(solver, num_flasks=22, flask_size=4, num_colors=20)

avg_sol_len = 0
avg_nodes_expanded = 0
avg_entropy = 0
avg_frag = 0
test_size = 100

for _ in tqdm(range(test_size)):
    logic = generator.generate()
    _, nodes_explored, path = solver.solve(logic)
    avg_sol_len += len(path)
    avg_nodes_expanded += nodes_explored
    avg_entropy += shannon_entropy(logic)
    avg_frag += fragmentation_score(logic)

avg_sol_len /= test_size
avg_nodes_expanded /= test_size
avg_entropy /= test_size
avg_frag /= test_size

print(avg_entropy, avg_frag, avg_nodes_expanded, avg_sol_len)
df.loc[len(df)] = {
    'algorithm': "Es", 
    'solution length': avg_sol_len,
    'nodes expanded': avg_nodes_expanded,
    'shannon entropy': avg_entropy,
    'fragmentation': avg_frag
}

100%|██████████| 100/100 [11:34<00:00,  6.95s/it]

1.4011186052147622 1.9595454545454547 3018.79 72.71


In [8]:
avg_sol_len = 0
avg_nodes_expanded = 0
avg_entropy = 0
avg_frag = 0
elites = 0

with open('map_es_flasks3.txt', 'r') as f:
    for line in f:
        line = line.replace('] [', '];[')

        flasks = [Flask(4, eval(x)) for x in line.split(';')]

        logic = Logic(flasks)
        _, nodes_explored, path = solver.solve(logic)
        avg_sol_len += len(path)
        avg_nodes_expanded += nodes_explored
        avg_entropy += shannon_entropy(logic)
        avg_frag += fragmentation_score(logic)

        elites += 1


avg_sol_len /= elites
avg_nodes_expanded /= elites
avg_entropy /= elites
avg_frag /= elites

print(avg_entropy, avg_frag, avg_nodes_expanded, avg_sol_len)
df.loc[len(df)] = {
    'algorithm': "MapElites", 
    'solution length': avg_sol_len,
    'nodes expanded': avg_nodes_expanded,
    'shannon entropy': avg_entropy,
    'fragmentation': avg_frag
}

1.6325335208230518 2.5814393939393936 3777.5833333333335 114.45833333333333


In [9]:
df.to_csv('results.csv')